# Limpieza de datos DimProducto

In [336]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Recolección de datos

In [337]:
df = pd.read_excel('/content/DimProducto.xlsx')

In [338]:
df.head(10)

,IdProducto,IdLinea,Nombre,completo,de,producto
0,11.0,1.0,GASA ESTERIL 10,10,100,NaN
1,19.0,2.0,TE ADELGAZANTE,NaN,NaN,NaN
2,20.0,2.0,TE DE MANZANILLA,NaN,NaN,NaN
3,21.0,2.0,TE RELAJANTE,NaN,NaN,NaN
4,23.0,2.0,TE ANTIGRIPAL,NaN,NaN,NaN
5,24.0,2.0,TE DE ZACATE LIMON,NaN,NaN,NaN
6,25.0,2.0,TE ROSA DE JAMAICA,NaN,NaN,NaN
7,30.0,2.0,LIPTOMIEL JARABE,NaN,NaN,NaN
8,31.0,2.0,LIPTOMINT JARABE,NaN,NaN,NaN
9,32.0,2.0,ENJUAGUE BUCAL ISNAYA,NaN,NaN,NaN


In [339]:
df.shape

(374, 6)

## Limpieza

In [340]:
df.columns

Index(['IdProducto', 'IdLinea', 'Nombre', 'completo', 'de', 'producto'], dtype='object')

### Concatenar columnas

In [341]:
# Nueva columna y concatenar columna con espacio (filtrando nulos)
df['Nombre'] = df[['Nombre', 'completo', 'de', 'producto']].astype(str).apply(lambda x: ' '.join([i for i in x if i != 'nan']), axis=1)
df = df.drop(columns=['completo', 'de', 'producto'])

In [342]:
df.head(10)

,IdProducto,IdLinea,Nombre
0,11.0,1.0,GASA ESTERIL 10 10 100
1,19.0,2.0,TE ADELGAZANTE
2,20.0,2.0,TE DE MANZANILLA
3,21.0,2.0,TE RELAJANTE
4,23.0,2.0,TE ANTIGRIPAL
5,24.0,2.0,TE DE ZACATE LIMON
6,25.0,2.0,TE ROSA DE JAMAICA
7,30.0,2.0,LIPTOMIEL JARABE
8,31.0,2.0,LIPTOMINT JARABE
9,32.0,2.0,ENJUAGUE BUCAL ISNAYA


### Reemplazos


In [344]:
# Diccionario de reemplazos de unidades
reemplazos_unidades = {
    'GRAMOS': 'G',
    'GR': 'G',
    'QUINCE': '15',
    'AMPOLLA': 'AMP',
    'CAP':'CAPS',
    'FRASCO':'FCO',
    'TABLETAS':'TAB',
    'CAPSULAS': 'CAPS',
    'MIL':'1000',
    'CAJA':'CJA',
    'NOVENTA':'90',
    'SESENTA': '60',
    'TRESCIENTOS': '300',
    'CINCUENTA': '50',
    'CINCO': '5',
    'TABLETA':'TAB',
    'JARABE': 'JBE',
    'UNIDADES':'UND',
    'UNIDAD':'UND',
    'UNDS':'UND',
    'CINCO':'5'


}

# Diccionario de reemplazos de términos
reemplazos = {
    'GUANTE DE E AMEN': 'GUANTE DE EXAMEN',
    'MUCOBRO OL': 'MUCROBROL',
    'O BRON': 'OXOBRON',
    'O OBRON': 'OXOBRON',
    'RELAFLE': 'RELAFLEX',
    'VIRO-GIP':'VIRO-GRIP',
    'TE TURIZADO':'TEXTURIZADO',
    'VIVE COLOR MI TO': 'VIVE COLORS MIXTO',
    'TRI OMEGA COMPLE': 'TRIPLE OMEGA COMPLEX',
    'CROTAMIN': 'CROTAMITON',
    'RARPEMA': 'RARPEMAX',
    'E PE T RAVEN COMPUESTO': 'EXPEXT RAVEN COMPUESTO',
    'DE AVITALGIA': 'DEXA-VITALGIA',
    'DE KETOPROFENO': 'DEXKETOPROFENO',
    'DE ADOCEPLE': 'DEXA-DOCEPLEX',
    'COBALE': 'COBALEX',
    'CELECO IB': 'CELECOBIX',
    'DOCEPLE': 'DOCEPLEX',
    'FE OFEN RAVEN': 'FEXOFEN RAVEN',
    'FORTIPLE': 'FORTIPLEX',
    'MYOTAN-': 'MYOTAN 50',
    '"D"':'D','"A"':'A',
    '"C"':'C', 'B-':'B',
    'DOLODOCEPLEX':'DOLO-DOCEPLEX',
    'TADALAFIL': 'TADALAFILO',
    'E-':'E','TESTE DEPOT':'TESTOVIRON DEPOT',
    'FENODOL MA':'FENODOL MAx', 'FE OFEN':'FEXOFEN',
    'VENOCELL COMPLE':'VENOCELL COMPLEX',
    'GANDE':'GRANDE',
    'GIPE':'GRIPE',
    'FENODOL MAx':'FENODOL MAX',
    'DEMISEL CLINDA VAGINAL AL': 'DEMISEL CLINDA VAGINAL',
    'GIPE Y TOS ACTIMICINA DIPS':'GRIPE Y TOS ACTIMICINA DIPS',
    '( FRESA )': '(FRESA)',
    '( SIMPLE )':'(SIMPLE)'
}


### Funciones de limpieza

In [345]:
import re
def obtener_base_producto(texto):
    texto = texto.upper()
    texto = re.sub(r'\s+', ' ', texto).strip()
    texto = re.sub(r'\*+', '', texto)
    base = texto.split('(')[0].strip()
    return base

Para el producto general (sin especificación)

In [346]:
def limpiar_producto_corte_en_numero(texto, flag):
    texto = texto.upper()
    texto = re.sub(r'\s+', ' ', texto).strip()
    texto = re.sub(r'\*+', '', texto)
    if '(E)' in texto:
        texto = texto.replace('(E)', 'E')
    if flag == 0:
      texto = re.sub(r'\(.*?\)', '', texto)
    texto = texto.strip()
    # Eliminar palabras residuales en cualquier parte del texto
    palabras_residuales = [
        'MG', 'ML', 'MCG', 'G', 'TAB', 'CAPS', 'FCO',
        'ROLLOS','SOFTGELS', 'CAPSULAS', 'SUSPENSION',
        'GOTAS', 'CREMA', 'GEL', 'INYECTABLE', 'LOCION', 'AMP', 'LIQUIDO',
        'SUSP.', 'ORAL', 'COMPOSITUM', 'OFERTA'
    ]
    pattern_residuales = r'\b(?:' + '|'.join(palabras_residuales) + r')\b'
    texto = re.sub(pattern_residuales, '', texto)


    # Aplicar reemplazos sobre el texto completo
    for clave, valor in reemplazos.items():
        texto = texto.replace(clave, valor)

    # Cortar en el primer número
    if flag == 0:
        match = re.search(r'\b\d+', texto)
        if match and 'RARPEZIT-600' not in texto:
            texto = texto[:match.start()].strip()
    else:
        # Solo si hay paréntesis
        if '(' in texto and ')' in texto:
            # Buscar número antes del paréntesis
            match = re.search(r'(\d+.*?)(?=\()', texto)
            if match:
                texto = texto.replace(match.group(1), '').strip()

    #Corte exclusivo: elimina todo después de la palabra, pero conserva la palabra
    palabras_corte_exclusivo = [
        'GRANDE', 'MEDIANO', 'PEQUEÑO'
    ]
    pattern_exclusivo = r'\b(' + '|'.join(palabras_corte_exclusivo) + r')\b.*'
    corte_exclusivo = re.search(pattern_exclusivo, texto)
    if corte_exclusivo:
        texto = texto[:corte_exclusivo.end(1)].strip()

    return texto.strip()


Para la especificación del producto

In [347]:
def extraer_especificacion(texto):
    texto = texto.upper()
    texto = re.sub(r'\s+', ' ', texto).strip()
    texto = re.sub(r'\*+', '', texto)
    texto = texto.replace(',', '')
    texto = re.sub(r'\bGR\b', 'G', texto)

    # Asegurar espacio entre número y letra
    texto = re.sub(r'(\d)(?=[A-Z])', r'\1 ', texto)

    # Detectar bloques tipo "10 CC 22 1.5"
    match = re.search(r'(\d+(?:\.\d+)?\s*CC(?:\s+\d+(?:\.\d+)?){1,2})', texto)
    if match:
        return match.group(1).strip()

    # Detectar tallas (de 6 a 9, opcionalmente con "1/2") ANTES de cualquier otro número
    match_talla = re.search(r'\b([6-9](?:\s*1/2)?)\b', texto)
    if match_talla:
        # Normalizamos espacio alrededor de la fracción
        return match_talla.group(1).replace('  ', ' ').strip()

    # Excepciones
    exceptions_list = ['ROCEFORT', 'DESKAN-C', 'DEKOFEN', 'ALTASTRESS SOLUCION']

    if any(ex in texto for ex in exceptions_list):
        # Buscar desde número principal hasta antes del paréntesis
       pattern = r'(\d[^\(\r\n]*?)(?=\s*\(|$)'
       match = re.search(pattern, texto)
       if match:
            return match.group(1).strip()

    # Unidades generales
    unidades = [
        'MG', 'ML', 'MCG', 'UND', 'CC', 'G', 'TAB', 'CAPS', 'JBE', 'CJA', 'FCO',
        'ROLLOS', 'SOFTGELS', 'GOTAS', 'CREMA', 'GEL', 'TUBO', 'SOLUCION', 'INYECTABLE', 'LOCION',
        'AMP', 'LIQUIDO', 'SUSP\\.?', 'SOBRES', 'BLISTER', 'PULG', 'PAQUETE'
    ]
    unidades_regex = '|'.join(unidades)

    if 'RARPEZIT-600' in texto or 'VIVE LUBRICANTE' in texto or 'VIVE COMBO PACK' in texto:
      return None

    pattern_general = rf'\b(\d+(?:\.\d+)?(?:/\d+)?(?:\s+(?:{unidades_regex})\b)?)'
    match_general = re.search(pattern_general, texto)
    if match_general:
        return match_general.group(1).strip()

    return None


In [348]:
df['Nombre'] = df['Nombre'].str.replace(',', '', regex=False)
df['Nombre'].replace(reemplazos_unidades, regex=True, inplace=True)

<ipython-input-348-b5f672da8a1c>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Nombre'].replace(reemplazos_unidades, regex=True, inplace=True)


### Comprobando estado actual

In [349]:
nombres_ordenados = sorted(df['Nombre'].unique())
for nombre in (nombres_ordenados):
    print(nombre)


ACEITE DE VITAMINA E RAVEN FCO   15 G
ACEITE HIGADO DE BACALAO FCO  100
ACEITE OMEGA 300MG   100 BLISTER (FARMACAPSS)
ACICLOVIR IMED SUSPENSIÓN FCO   120 ML (QUIMIFAR)
ACIDO FOLICO 1 MG   30 TAB (RAVEN)
ACTIMICINA BRONQUIAL- F   72 TAB (RARPE)
AJOCAPSS CAPSSULAS 350MG   100 BLISTER (FARMACAPSS)
AJOCAPSS CON PEREJIL 200MG 100MG CJA   100 BLISTER (FARMACAPSS
ALGODÓN 1 LIBRA
ALGODÓN 1/2 LIBRA
ALLIVIA  CON NEUROTROPAS   100 GELCAPSS (FARMACAPSS)
ALTACIDEZ SUSPENSION FCO 360ML (ALTASA)
ALTAGIP CJA   60 TAB (ALTASA)
ALTAGIP GOTAS 30ML (ALTASA)
ALTAGIP JBE 120 ML (ALTASA)
ALTAGOLPE GEL TUBO 30 G (ALTASA)
ALTAGOLPE SOLUCION FCO 100 ML (ALTASA)
ALTAGOLPE SOLUCION FCO 100 ML (BANDEADO SIN DEVOLUCION)
ALTAHONGO CREMA 15G (ALTASA)
ALTALOR 500MG CJA   60 TAB (ALTASA)
ALTALOR 500MG CJA   60 TAB (BANDEADO SIN DEVOLUCION)
ALTALOR GOTAS 30ML (ALTASA)
ALTALOR JBE FCO 120ML (ALTASA)
ALTALOR RELA  CJA   30 TAB (ALTASA)
ALTAMENSTRUAL 400MG   60 TAB (ALTASA)
ALTAMENSTRUAL 400MG   60 TAB (BANDEADO SIN DEVOL

### Aplicando las funciones

In [350]:
df['base_producto'] = df['Nombre'].apply(obtener_base_producto)

# Contar cuántas veces aparece cada base
conteos = df['base_producto'].value_counts()

# Crear flag: 1 si hay más de una versión del mismo producto base, 0 si es único
excepciones = ['DEKOFEN 25MG', 'ALTASTRESS SOLUCION']

df['flag'] = df['base_producto'].map(
    lambda x: 1 if (conteos[x] > 1 or any(x.startswith(exc + ' ') or x == exc for exc in excepciones)) else 0
)

# Eliminar columna temporal
df.drop(columns='base_producto', inplace=True)

df.head(200)

,IdProducto,IdLinea,Nombre,flag
0,11.0,1.0,GASA ESTERIL 10 10 100,0
1,19.0,2.0,TE ADELGAZANTE,0
2,20.0,2.0,TE DE MANZANILLA,0
3,21.0,2.0,TE RELAJANTE,0
4,23.0,2.0,TE ANTIGIPAL,0
5,24.0,2.0,TE DE ZACATE LIMON,0
6,25.0,2.0,TE ROSA DE JAMAICA,0
7,30.0,2.0,LIPTOMIEL JBE,0
8,31.0,2.0,LIPTOMINT JBE,0
9,32.0,2.0,ENJUAGUE BUCAL ISNAYA,0


In [352]:
# Extraer especificación
df['Especificacion'] = df['Nombre'].apply(extraer_especificacion)

# Aplicar limpieza de nombre
df['Nombre Producto'] = df.apply(lambda row: limpiar_producto_corte_en_numero(row['Nombre'], row['flag']), axis=1)

###Reordenando columnas

In [353]:
df = df[['IdProducto','IdLinea', 'Nombre Producto', 'Especificacion']]
pd.set_option('display.max_rows', None)
df.head(400).sort_values(by='Nombre Producto')

,IdProducto,IdLinea,Nombre Producto,Especificacion
373,2864.0,NaN,,None
70,NaN,NaN,,None
372,2863.0,NaN,,None
367,2858.0,NaN,,None
62,NaN,NaN,,None
369,2860.0,NaN,,None
370,2861.0,NaN,,None
371,2862.0,NaN,,None
368,2859.0,NaN,,None
299,2766.0,11.0,ACEITE DE VITAMINA E RAVEN,15 G


### Eliminando nulos

In [354]:
df['Especificacion'].fillna('', inplace=True)

<ipython-input-354-34960fc4fd8b>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Especificacion'].fillna('', inplace=True)
<ipython-input-354-34960fc4fd8b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Especificacion'].fillna('', inplace=True)


In [355]:
df.isnull().sum()

,0
IdProducto,2
IdLinea,9
Nombre Producto,0
Especificacion,0


In [356]:
print(df[df['IdLinea'].isnull()])
index_drop = list(df[df['IdLinea'].isnull()].index)

     IdProducto  IdLinea Nombre Producto Especificacion
62          NaN      NaN                               
70          NaN      NaN                               
367      2858.0      NaN                               
368      2859.0      NaN                               
369      2860.0      NaN                               
370      2861.0      NaN                               
371      2862.0      NaN                               
372      2863.0      NaN                               
373      2864.0      NaN                               


In [357]:
index_drop

[62, 70, 367, 368, 369, 370, 371, 372, 373]

In [358]:
# eliminar nulos en IdLinea
df = df.dropna(subset=['IdLinea'])

In [359]:
df.isnull().sum()

,0
IdProducto,0
IdLinea,0
Nombre Producto,0
Especificacion,0


In [360]:
df['NombreProducto'] = df['Nombre Producto'] + ' ' + df['Especificacion']
df.head(10)

<ipython-input-360-e033dcf43490>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NombreProducto'] = df['Nombre Producto'] + ' ' + df['Especificacion']


,IdProducto,IdLinea,Nombre Producto,Especificacion,NombreProducto
0,11.0,1.0,GASA ESTERIL,10,GASA ESTERIL 10
1,19.0,2.0,TE ADELGAZANTE,,TE ADELGAZANTE
2,20.0,2.0,TE DE MANZANILLA,,TE DE MANZANILLA
3,21.0,2.0,TE RELAJANTE,,TE RELAJANTE
4,23.0,2.0,TE ANTIGIPAL,,TE ANTIGIPAL
5,24.0,2.0,TE DE ZACATE LIMON,,TE DE ZACATE LIMON
6,25.0,2.0,TE ROSA DE JAMAICA,,TE ROSA DE JAMAICA
7,30.0,2.0,LIPTOMIEL JBE,,LIPTOMIEL JBE
8,31.0,2.0,LIPTOMINT JBE,,LIPTOMINT JBE
9,32.0,2.0,ENJUAGUE BUCAL ISNAYA,,ENJUAGUE BUCAL ISNAYA


In [361]:
df = df.drop(columns=['Nombre Producto', 'Especificacion'])
df.head(10)

,IdProducto,IdLinea,NombreProducto
0,11.0,1.0,GASA ESTERIL 10
1,19.0,2.0,TE ADELGAZANTE
2,20.0,2.0,TE DE MANZANILLA
3,21.0,2.0,TE RELAJANTE
4,23.0,2.0,TE ANTIGIPAL
5,24.0,2.0,TE DE ZACATE LIMON
6,25.0,2.0,TE ROSA DE JAMAICA
7,30.0,2.0,LIPTOMIEL JBE
8,31.0,2.0,LIPTOMINT JBE
9,32.0,2.0,ENJUAGUE BUCAL ISNAYA


In [362]:
df.to_excel('DimProductoLimpio.xlsx', index=False)

In [363]:
df

,IdProducto,IdLinea,NombreProducto
0,11.0,1.0,GASA ESTERIL 10
1,19.0,2.0,TE ADELGAZANTE
2,20.0,2.0,TE DE MANZANILLA
3,21.0,2.0,TE RELAJANTE
4,23.0,2.0,TE ANTIGIPAL
5,24.0,2.0,TE DE ZACATE LIMON
6,25.0,2.0,TE ROSA DE JAMAICA
7,30.0,2.0,LIPTOMIEL JBE
8,31.0,2.0,LIPTOMINT JBE
9,32.0,2.0,ENJUAGUE BUCAL ISNAYA
